In [1]:
import explicit_statetespace as efss
import ssr_kronecker_vector as ssr
import ssr_likelihood as ssr_lklhd
import likelihood as fss_lklhd
import kronecker_vector as fss
import numpy as np
import Utilityfunctions as utils


In [2]:
n = 2
sparsity = 0.5
log_theta = utils.random_theta(n=n, sparsity=sparsity)


In [3]:
state = np.random.randint(2, size=2*n+1)
length = 2**sum(state)
state


array([0, 1, 1, 0, 1])

In [4]:
mut = [state[j] + 2 * state[j + 1] for j in range(0, 2 * n, 2)]
mut.append(state[-1])
mut = np.array(mut)


In [5]:
# Test explicit ssr vs. shuffle trick ssr
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        efss.ssr_build_q(dpoint=state, log_theta=log_theta) @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    ))


In [6]:
# Test no diag
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    q = ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    q[j] = 0
    assert (np.allclose(
        q,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state, diag=False)
    ))


In [7]:
# Test diag
assert (np.allclose(
    np.diag(efss.ssr_build_q(dpoint=state, log_theta=log_theta)),
    ssr.kron_diag(log_theta=log_theta, n=n, state=state)
))


In [8]:
# Test R inverse
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        np.linalg.inv(np.identity(
            length) - efss.ssr_build_q(dpoint=state, log_theta=log_theta)) @ p,
        ssr_lklhd.R_i_inv_vec(log_theta=log_theta, x=p, lam=1, state=state),
    ))


In [9]:
# Test transpose
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert (np.allclose(
        efss.ssr_build_q(dpoint=state, log_theta=log_theta).T @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state, transpose=True)
    ))


In [10]:
# Test ssr vs fss
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    index = utils.ssr_to_fss(state)
    assert (np.allclose(
        efss.build_q(log_theta=log_theta)[np.ix_(index, index)] @ p,
        ssr.kronvec(log_theta=log_theta, p=p, n=n, state=state)
    ))


In [5]:
# Test log_likelihood

lam1, lam2 = np.random.random(2)
reachable = utils.reachable_states(n)
for j in range(2**(2*n+1)):
    if not reachable[j]:
        continue
    p = np.zeros(2**(2*n+1))
    p[j] = 1
    np.allclose(
        fss_lklhd.log_likelihood(
            log_theta=log_theta, pd=p, lam1=lam1, lam2=lam2),
        ssr_lklhd.log_likelihood(log_theta=log_theta, p_D=p, lam1=lam1, lam2=lam2, state=state))


In [6]:
# Test gradient

for j in range(2**(2*n+1)):
    if not reachable[j]:
        continue
    p = np.zeros(2**(2*n+1))
    p[j] = 1
    ssr_lklhd.gradient(log_theta=log_theta, p_D=p[utils.ssr_to_fss(state)],
                       lam1=lam1, lam2=lam2, state=state)
